In [1]:
from pyscf import gto,scf
import pyscf
import matplotlib.pyplot as plt
from scipy.interpolate import InterpolatedUnivariateSpline as spl
import numpy as np
from FcMole import FcM
ang2bohr=1.8897261246
bohr2ang=.5291772109

In [11]:
CO=gto.M(atom="C 0 0 0; O 0 0 2.05",unit='Bohrs',basis="pc4",verbose=3)
mf=scf.RHF(CO)
CO.nao

218

In [12]:
def optimizer2at(a1,a2,fcs=None):
    d1=np.linspace(1.9,2.4,9)
    es=[]
    for d in d1:
        if fcs==None:
            mol=gto.M(atom="{} 0 0 0; {} 0 0 {} ".format(a1,a2,d),unit='Bohrs',basis="pc4",verbose=0)
            mf=scf.RHF(mol)
            es.append(mf.scf())
        else:
            mol=FcM(fcs=fcs,atom="{} 0 0 0; {} 0 0 {} ".format(a1,a2,d),unit='Bohrs',basis="pc4",verbose=0)
            mf=scf.RHF(mol)
            es.append(mf.scf(dm0=mf.init_guess_by_1e()))
    dspl=np.linspace(1.9,2.4,501)
    bspline=spl(d1,es)
    bspline(dspl)
    dminspline=1.9+.001*(bspline(dspl).argmin())
    d2=np.linspace(dminspline-.03,dminspline+.03,7)
    es=[]
    for d in d2:
        if fcs==None:
            mol=gto.M(atom="{} 0 0 0; {} 0 0 {} ".format(a1,a2,d),unit='Bohrs',basis="pc4",verbose=0)
            mf=scf.RHF(mol)
            es.append(mf.scf())
        else:
            mol=FcM(fcs=fcs,atom="{} 0 0 0; {} 0 0 {} ".format(a1,a2,d),unit='Bohrs',basis="pc4",verbose=0)
            mf=scf.RHF(mol)
            es.append(mf.scf(dm0=mf.init_guess_by_1e()))
    dspl=np.linspace(dminspline-.03,dminspline+.03,6001)
    bspline=spl(d2,es)
    #plt.plot(dspl,bspline(dspl))
    COmin=dspl[bspline(dspl).argmin()]
    return COmin,bspline(dspl).min()

In [13]:
optimizer2at("C","O")

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/giorgiod/miniconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3417, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-13-9b3551d5fc86>", line 1, in <module>
    optimizer2at("C","O")
  File "<ipython-input-12-8b1a6729367c>", line 8, in optimizer2at
    es.append(mf.scf())
  File "/home/giorgiod/miniconda3/lib/python3.7/site-packages/pyscf/scf/hf.py", line 1644, in scf
    conv_check=self.conv_check, **kwargs)
  File "/home/giorgiod/miniconda3/lib/python3.7/site-packages/pyscf/scf/hf.py", line 177, in kernel
    vhf = mf.get_veff(mol, dm, dm_last, vhf)
  File "/home/giorgiod/miniconda3/lib/python3.7/site-packages/pyscf/scf/hf.py", line 1960, in get_veff
    vj, vk = self.get_jk(mol, dm, hermi)
  File "/home/giorgiod/miniconda3/lib/python3.7/site-packages/pyscf/scf/hf.py", line 1950, in get_jk
    vj, vk = dot_eri_dm(self._eri, dm, hermi, with_j, with_k)
  File "/home/giorgio

TypeError: object of type 'NoneType' has no len()

In [ ]:
optimizer2at("N","N")

In [ ]:
optimizer2at("B","F")

# GAUGE THE ERROR DUE TO BASIS SET

In [ ]:
#CO@NN
optimizer2at("N","N",fcs=[-1,1.])

In [ ]:
#CO@BF
optimizer2at("B","F",fcs=[1.,-1.])

In [ ]:
#BF@CO
optimizer2at("C","O",fcs=[-1,1])

In [ ]:
#NN@CO
optimizer2at("C","O",fcs=[1,-1])